# Gereksinim Kontrolleri

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print("GPU'ya bağlı olmamanız nedeniyle işleme devam edilemez.")
else:
  print(gpu_info)

Mon May 27 12:39:39 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0              44W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

# Gerekli kütüphanelerin kurulumu ve projeye dahil edilmesi

In [2]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps trl peft accelerate bitsandbytes
!pip install xformers[cuda]
!pip install wandb

import pandas as pd
import torch

from datasets import Dataset, DatasetDict
from unsloth import FastLanguageModel

from trl import SFTTrainer
from transformers import TrainingArguments
from datasets import load_dataset

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-20kyfa1c/unsloth_774902824f3e44e39382cae5d3f8d01f
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-20kyfa1c/unsloth_774902824f3e44e39382cae5d3f8d01f
  Resolved https://github.com/unslothai/unsloth.git to commit b0781339f035c72b3028d846eb2261e8115cd375
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.2 MB/s eta 0:00:00
  Created wheel for un

# LLM Modelinin Yüklenmesi ve Başlangıç Ayarları


In [3]:
max_seq_length = 2048 # Modele girecek metin girdisinin maksimum uzunluğunu ifade eder. 2048 token olarak tanımladık.
dtype = None # Unslothda kullanacağımız veri türünü belirleyebiliriz. None olarak bırakırsak varsayılan float32 olarak işlem yapar.
load_in_4bit = True # Model ağırlığını 4 bit hassasiyetle yüklemek isteyip istemediğimizi belirtiriz. Bu işlem bellek kullanımını önemli ölçüde azaltır fakat modelin doğruluğunu biraz etkileyebilir.

# FastLanguageModel.from_pretrained() : Önceden eğtilmiş olan bir LLM modelini yüklüyoruz.
# Model_name alanında 8 milyar parametreli llama-3 modelinin 4 bit ağırlıklı versiyonunu yükleyeceğimizi belirtiyoruz.
# max_seq_length, dtype, load_in_4bit alanlarına ise yukarıda tanımladığımız değişken verilerini atıyoruz.
# Kısacası aşağıdaki kod, belirtilen model adını, maksimum dizi uzunluğunu, veri türünü ve 4 bit yükleme seçeneğini kullanarak FashLanguageModel ->
# -> sınıfından nesne oluşturur. Bu nesne modeli ve metin verilerini tokenlara dönüştürmek için kullanılacak bir tokenizer içerir.
# tokenizer ve token ne demek? Bu yazılacak.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit", #8 milyar parametreli llama-3 modelinin 4 bit ağırlıklı versiyonunu yükleyeceğimizi belirtiyoruz.
    max_seq_length = max_seq_length, #modelin işleyebileceği en uzun dizinin (kelime veya token dizisi) uzunluğunu belirtir. Dizinin uzunluğu, modelin bir seferde işleyebileceği maksimum karakter veya kelime sayısını ifade eder.
    dtype = dtype, # Modelin kullanacağı veri tipini belirtir.
    load_in_4bit = load_in_4bit # Modelin 4-bit ağırlıkla yüklenip yüklenmeyeceğini belirtir. Bu, modelin hafıza kullanımını azaltarak daha az yer kaplamasını ve bazı durumlarda daha hızlı çalışmasını sağlar.
)

config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Llama patching release 2024.5
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. Xformers = 0.0.26.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/464 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# LLM Modelinin PEFT Ayarlarının Yapılandırılması


In [4]:
# PEFT : Positional Encoding Fine-Tuning
# Bu teknik önceden eğitilmiş bir dil modelinin konumsal kodlamasını bir veri seti için özelleştirmeyi sağlar. Daha iyi sonuçlar elde etmek için modelin ağırlıklarını ayarlama işlemini de yapar. Konumsal kodlama, bir cümledeki kelimelerin sırasını modelin anlamasına yardımcı olan bir tekniktir.
# LoRA : Low-Rank Adaptation
# LoRA Gelişmiş bir ince ayar yöntemidir. Hesaplama maliyetini düşürür ve modelin aşırı öğrenmesini önlemeye de yardımcı olur.

# Unsloth kütüphanesinin PEFT ve LoRA özelliklerini kullanarak önceden eğitilmiş bir LLM modelini daha verimli ve göreve özel hale getirmeyi sağlar. Bu teknikler, modelin boyutunu küçültür ve bellek kullanımını azaltır. Böylece performansı artırır.

model = FastLanguageModel.get_peft_model(
    model,
    r = 16,

    #target_modules dizisi, PEFT ve LoRA ile özelleştirilmek istenen modülleri liste olarak tutar. Bu modüller modelin dikkat mekanizmasında ve konumsal kodlamada kullanılacaktır.

    # q_proj: Sorgu projeksiyonu: Diğer tokenlarla olan ilişkilerin değerlendirilmesi için kullanılır.
    # k_proj: Anahtar projeksiyonu: Anahtarlar, sorguların hangi bilgilere odaklanacağını belirlemek için kullanılır.
    # v_proj: Değer projeksiyonu: Girdi verisinin "değer" vektörlerine dönüştürülmesini sağlar.
    # o_proj: Çıkış projeksiyonu: Vektörlerin birleştirilmesi ve dönüştürülmesi işlemlerini kapsar.
    # gate_proj: Kapı projeksiyonu: Bir nevi bilgi akışını kontrol eden bir kapı görevi görür. Hangi bilgilerin ileriye aktarılacağını veya dışlanacağını belirler.
    # up_proj: Yükseltme projeksiyonu: Düşük boyutlu girdileri daha yüksek boyutlu bir uzaya taşıyarak modelin daha fazla bilgiyi işlemesini sağlar.
    # down_proj: Düşürme projeksiyonu: Yüksek boyutlu girdileri daha düşük boyutlu bir uzaya indirgeyerek hesaplama maliyetini azaltır.
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],

    # lora_alpha: Daha yüksek alfa değeri, daha güçlü doğrusallaştırma ve potansiyel olarak daha iyi performans anlamına gelir.
    lora_alpha = 16,

    # lora_dropout: LoRA için dropout oranını belirler. Dropout, aşırı öğrenmeyi önlemek için rastgele nöronları devre dışı bırakır. 0 değeri optimize edilmiş performans için önerilir.
    lora_dropout = 0,

    # bias: "none" değeri, optimize edilmiş performans için önerilir.
    bias = "none",

    # use_gradient_checkpointing: Bellek verimliliği için gradyon kontrol noktasını etkinleştirir. Bellek kullanımını azaltmak için unsloth veya True değerini kullanabiliriz.
    use_gradient_checkpointing = "unsloth",

    # random_state: Yeniden üretilebilirlik için rastgele bir durum belirler. Aynı rastgele durumları kullanarak işlemleri tekrarlamayı sağlayabiliriz.
    random_state = 3407,

    # use_rslora: Rütbe sabitli LoRA'yı etkinleştirir veya devre dışı bırakır. - Modelin ağırlık matrislerini değiştirmek yerine bu ağırlıklara ek dönüşüm matrisleri eklenmesi ile ilgilidir.
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2024.5 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


# Veri Setinin Yüklenmesi ve Veri Setinin Talimat Şablonuna Entegrasyonu


In [5]:
#Input: Örnek veriyorum mesela instruction'a : Sıradaki örüntüyü tamamla diyip, input'a 1,2,4,8,16 diyebiliriz.
# instruction konuyu ifade eder.
# output için ise cevap veya çözümü veya konunun açıklamasını ifade eder.

def formattingPrompts(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []

    for instruction, input, output in zip(instructions, inputs, outputs):
        text = nixous_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

# Belirli bir görevi tanımlayan talimatlarla, bu talimatları tamamlayacak girdileri ve bunlara uygun yanıtları eşleştirmek için kullanılır. Bu şablon, yapay zeka modelinin, verilen talimatı anlamasını ve bu talimata uygun olarak verilen girdiye dayanarak doğru bir yanıt üretmesini sağlamak üzere tasarlanmıştır.
nixous_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Bir cümlenin veya girdi dizisinin sona erdiğini belirten bir işaretçidir.

from google.colab import drive
drive.mount('/content/drive')
excel_data = pd.read_excel("/content/drive/MyDrive/new_train.xlsx")
dataset = Dataset.from_pandas(excel_data)
dataset = dataset.map(formattingPrompts, batched=True)

Mounted at /content/drive


Map:   0%|          | 0/797739 [00:00<?, ? examples/s]

# Veri Setinin Analizi


In [6]:
print(dataset)

Dataset({
    features: ['instruction', 'input', 'output', 'text'],
    num_rows: 797739
})


# Dil Modelinin Eğitim Ayarları ve Fine-Tune işleminin Gerçekleştirilmesi


In [7]:

# Bu kodun ana görevi, belirtilen parametreler ve yapılandırmalar doğrultusunda bir dil modelinin
# eğitim sürecini başlatmak ve yönetmektir. Eğitim sırasında, model verilen veri seti üzerinde eğitilir
# ve belirtilen ayarlar (örneğin, öğrenme hızı, adım sayısı) doğrultusunda optimize edilir.
# Bu süreç, modelin belirli bir doğal dil işleme görevinde daha iyi performans göstermesi için gereklidir.
# Modelin eğitimi, veri setinden öğrenilen özellikler ve bağlamlar ile geliştirilir, böylece model daha etkili ve doğru tahminler yapabilir.
# Eğitim sonuçları, ileriye dönük uygulamalar veya daha detaylı analizler için outputs dizinine kaydedilir.
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,

    args = TrainingArguments(
        report_to="wandb", # Eğitim istatistiklerinin raporlanacağı hedefleri belirtir.
        per_device_train_batch_size = 2, # Her bir cihazda (GPU/CPU) kullanılacak eğitim örnek sayısını belirtir.
        gradient_accumulation_steps = 4, # Gradyan toplama adımlarını belirtir; bu, daha büyük etkili toplu iş boyutları kullanabilmek için küçük toplu işler üzerinde gradyanların toplanmasını sağlar.
        warmup_steps = 5, # Eğitim başlangıcında öğrenme hızını kademeli olarak artırmak için kullanılan adımların sayısını belirtir.
        #num_train_epochs = 2,
        max_steps = 2000, # Eğitimin maksimum adım sayısını belirtir. num_train_epochs yerine kullanıldığında, eğitim belirtilen adım sayısına ulaştığında durur.
        learning_rate = 2e-4, # Öğrenme hızını belirtir.
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1, # Her kaç adımda bir log kaydı yapılacağını belirtir.
        optim = "adamw_8bit", # Kullanılacak optimizasyon algoritmasını belirtir.
        weight_decay = 0.01, # Ağırlık azaltma değerini belirtir, bu da aşırı uyumu önlemeye yardımcı olur.
        lr_scheduler_type = "linear", # Öğrenme hızı zamanla nasıl değişeceğini belirten zamanlayıcı tipini ifade eder. Çeşitleri: TODO
        seed = 3407,
        #do_eval=True,
        #eval_steps=100,
        output_dir = "outputs", # Eğitim sonuçlarının kaydedileceği dizini belirtir.
    ),
)

/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=2):   0%|          | 0/797739 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [8]:
# ÖNEMLİ! 64 işlem sonrası colab bağlantısı kesildi ve alt satırında tekrardan eğitmeye başladı.
# Bu sebeple eğitim istatistikleri ve tamamlanma oranı 64 satır aşağıdadır hocam.
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 797,739 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 2,000
 "-____-"     Number of trainable parameters = 41,943,040
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
1,1.387700
2,1.494000
3,1.384700
4,1.457000
5,1.558300
6,1.300800
7,1.416400
8,1.186000
9,1.237200
10,1.128300


Step,Training Loss
1,1.387700
2,1.494000
3,1.384700
4,1.457000
5,1.558300
6,1.300800
7,1.416400
8,1.186000
9,1.237200
10,1.128300


# Tokenizer İşlemleri


In [9]:
# Dil modelleri, doğrudan ham metin üzerinde çalışamazlar. Bunun yerine, metni anlamlı bir şekilde işleyebilmek için metinleri öncelikle daha küçük birimlere ve sayısal değerlere ayırmaları gerekir.
# Bu işlem tokenizer tarafından yapılır ve "tokenization" olarak adlandırılır.
# Tokenization işleminin birkaç önemli nedeni vardır:
# 1. Standart Format: Modelin, eğitim sürecinde öğrendiği token formatı ile uyumlu girdiler alması gerekir. Eğitim sırasında model, belirli token dizilimlerini öğrenir ve bu dizilimlere göre tahminlerde bulunur.
# 2. Verimlilik: Tokenlar, dilin yapısal öğelerini temsil eder. Model, bu yapısal öğeleri kullanarak metni daha hızlı ve etkin bir şekilde işleyebilir. Örneğin, bazı yaygın kelimeler veya ifadeler tek bir token olarak temsil edilebilir, bu da modelin daha az işlem yaparak daha hızlı sonuçlar vermesini sağlar.
# 3. Anlamı Yakalama: Dilin karmaşıklığını ve anlamını daha iyi yakalamak için bazen kelimelerin kökleri veya morfolojik varyantları gibi daha derin dilbilimsel öğeler token olarak kullanılır. Bu, modelin dilin nüanslarını ve anlam katmanlarını daha iyi anlamasına olanak tanır.
# 4. Bellek ve İşlem Kapasitesi: Eğer model her bir karakter üzerinden çalışsaydı, çok daha fazla bilgiyi işlemesi ve saklaması gerekecekti. Tokenization sayesinde, model yalnızca anlamlı tokenlere odaklanır, bu da hem bellek kullanımını hem de işlem yükünü azaltır.

# Mevcut dil modelini doğrudan çıktı üretmek için optimize eder. Bu, modelin işlem süresini kısaltır ve daha hızlı sonuç alınmasını sağlar.
FastLanguageModel.for_inference(model)

# Burada bir metin listesi, modelin anlayabileceği bir forma (token'lar haline) dönüştürülür. Bu işlem sırasında nixous_prompt.format(...) fonksiyonu kullanılarak bir format şablonundan yararlanılır.
# Bu format şablonu, özellikle bir talimat verilerek, girdi ve çıktı alanları boş bırakılarak doldurulmuştur.
inputs = tokenizer(
[
    nixous_prompt.format(
        # Aşağıya soru yazılacak.
        "How do I use components to display my dynamic navigation bar using laravel?",
        "", # input - Instructiona bana vereceğim sayıları çarpar mısın yazmamız durumunda inputa 2x2 verebiliriz.
        "", # output - Boş bırakılmalıdır.
    )
], return_tensors = "pt").to("cuda") # return_tensors = "pt": Token'lar, PyTorch tensörlerine dönüştürülür.

# .to("cuda"): Elde edilen tensörler, CUDA uyumlu bir cihazda (genellikle bir GPU) işlenmek üzere gönderilir. Bu, işlemlerin çok daha hızlı yapılmasını sağlar.

# model.generate(...): Modelin, verilen girdilere dayanarak yeni metin üretmesi sağlanır.
# **inputs: Tokenizer tarafından oluşturulan girdi tensörleri.
# max_new_tokens = 64: Modelin, en fazla 64 yeni token (kelime veya kelime parçaları) üretebileceğini belirtir.
# use_cache = True: Modelin, hesaplamaları daha verimli yapmak için önceki hesaplamaların sonuçlarını önbellekte tutmasına izin verir.
outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)

print("outputs:")
print(outputs)

print("input:")
print(inputs)
# Model tarafından üretilen çıktılar (token'lar), insanların anlayabileceği düz metinlere dönüştürülür. Bu işlem, tokenizer'ın batch_decode fonksiyonu ile gerçekleştirilir.
tokenizer.batch_decode(outputs)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


outputs:
tensor([[128000,  39314,    374,    459,   7754,    430,  16964,    264,   3465,
             11,  35526,    449,    459,   1988,    430,   5825,   4726,   2317,
             13,   9842,    264,   2077,    430,  36001,  45695,    279,   1715,
            382,  14711,  30151,    512,   4438,    656,    358,   1005,   6956,
            311,   3113,    856,   8915,  10873,   3703,   1701,  45555,   3963,
           1980,  14711,   5688,  25393,  14711,   6075,    512,   8085,     29,
           4599,    499,    527,   1701,  65306,    220,     20,     13,     20,
            477,   3485,     11,    499,    649,   1005,    366,   1889,     29,
           5004,    487,   1068,  87761,   1889,     29,   1749,    311,    471,
            264,   1684,     13,    694,     79,     29,    220,    366,     79,
             29,   2520,   3187,   7920,     79,     29,    220,    366,   1762,
           1822,   1889,     29,   5004,    487,   1068,   3478,   8134,    518,
            364,   

["<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nHow do I use components to display my dynamic navigation bar using laravel?\n\n### Input:\n\n\n### Response:\n<p>When you are using Laravel 5.5 or above, you can use <code>Route::view()</code> method to return a view. </p>  <p>For example:</p>  <pre><code>Route::view('/foo', 'foo'); </code></pre>"]

# TextStreamer ile Sonucun Gösterilmesi


In [10]:
# Mevcut dil modelini doğrudan çıktı üretmek için optimize eder. Bu, modelin işlem süresini kısaltır ve daha hızlı sonuç alınmasını sağlar.
FastLanguageModel.for_inference(model) #2x faster

# inputs = tokenizer([...], return_tensors = "pt").to("cuda"): Metni, model tarafından işlenebilecek token'lara çevirir. Bu, modelin metni anlamasına ve üzerinde işlem yapmasına olanak tanır.
#
inputs = tokenizer(
[
    nixous_prompt.format(
        # Soru aşağıya yazılacak.
        "How do I use components to display my dynamic navigation bar using laravel?", # instruction
         "",# input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer

# TextStreamer'ın ana görevi, metin üretilirken de ekrana yazdırmak. Örnek chatGPT cevap yazarken harfler
# tek tek ekrana düşerken, gemini çıktı üretildikten sonra ekrana basar.
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 1024)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
How do I use components to display my dynamic navigation bar using laravel?

### Input:


### Response:
<p>For anyone who is looking for a good way to create dynamic menus in Laravel, I would recommend using <a href="https://github.com/illuminate/html" rel="nofollow noreferrer">Laravel's HTML package</a> which is included in Laravel 5.2 and above.</p>  <p>It's very easy to use and makes it easy to create menus, tables, forms and more.</p>  <p>For example, to create a menu you can do:</p>  <pre><code>$menu = Menu::make('My Menu'); $menu-&gt;add(['Link text', 'link']); $menu-&gt;add(['Link text', 'link', ['class' =&gt; 'class', 'target' =&gt; '_blank']]); </code></pre>  <p>Or to create a table you can do:</p>  <pre><code>$table = Table::make(); $table-&gt;addColumns(['Name', 'Email', 'Role', 'Actio

# Eğitilen Modelin Export Edilmesi


In [11]:
model.save_pretrained("nixous_model")
tokenizer.save_pretrained("nixous_model")

('nixous_model/tokenizer_config.json',
 'nixous_model/special_tokens_map.json',
 'nixous_model/tokenizer.json')

# Eğitilen Model Üzerinden Sonuç Yazdırma


In [12]:
from unsloth import FastLanguageModel
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "./nixous_model",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)
FastLanguageModel.for_inference(model)

inputs = tokenizer(
[
    nixous_prompt.format(
        "How do I use components to display my dynamic navigation bar using laravel?", # instruction
        "", # input
        "", # output
    )
], return_tensors = "pt").to("cuda")


outputs = model.generate(**inputs, max_new_tokens=500, use_cache=True)
response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
response = response.split("### Response:\n")[1].strip()

print(response)

==((====))==  Unsloth: Fast Llama patching release 2024.5
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. Xformers = 0.0.26.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<p>From what I understand you want to create a dynamic navigation bar, you can use the <a href="https://laravel.com/docs/5.8/blade#template-inheritance" rel="nofollow noreferrer">template inheritance</a> to create the layout of your website and then use the <a href="https://laravel.com/docs/5.8/blade#the-component-tag" rel="nofollow noreferrer">component</a> tag to insert the navigation bar into your layout.</p>  <p>For example:</p>  <pre><code>&lt;!-- /resources/views/layouts/app.blade.php --&gt; &lt;html&gt;     &lt;head&gt;         &lt;title&gt;App Name - @yield('title')&lt;/title&gt;     &lt;/head&gt;      &lt;body&gt;         @yield('content')     &lt;/body&gt; &lt;/html&gt; </code></pre>  <p>Then you can create a component for the navigation bar and insert it into the layout:</p>  <pre><code>&lt;!-- /resources/views/components/navbar.blade.php --&gt; &lt;ul&gt;     &lt;li&gt;&lt;a href="/"&gt;Home&lt;/a&gt;&lt;/li&gt;     &lt;li&gt;&lt;a href="/about"&gt;About&lt;/a&gt;&lt;/li&gt

# Dosyaların ziplenmesi ve çıktılarının alınması


In [14]:
!zip -r huggingface_tokenizers_cache.zip /content/huggingface_tokenizers_cache

  adding: content/huggingface_tokenizers_cache/ (stored 0%)
  adding: content/huggingface_tokenizers_cache/models--unsloth--llama-3-8b-bnb-4bit/ (stored 0%)
  adding: content/huggingface_tokenizers_cache/models--unsloth--llama-3-8b-bnb-4bit/.no_exist/ (stored 0%)
  adding: content/huggingface_tokenizers_cache/models--unsloth--llama-3-8b-bnb-4bit/.no_exist/90bd376f78b5ba9ad646082570a99a33801c99ef/ (stored 0%)
  adding: content/huggingface_tokenizers_cache/models--unsloth--llama-3-8b-bnb-4bit/.no_exist/90bd376f78b5ba9ad646082570a99a33801c99ef/added_tokens.json (stored 0%)
  adding: content/huggingface_tokenizers_cache/models--unsloth--llama-3-8b-bnb-4bit/refs/ (stored 0%)
  adding: content/huggingface_tokenizers_cache/models--unsloth--llama-3-8b-bnb-4bit/refs/main (deflated 3%)
  adding: content/huggingface_tokenizers_cache/models--unsloth--llama-3-8b-bnb-4bit/blobs/ (stored 0%)
  adding: content/huggingface_tokenizers_cache/models--unsloth--llama-3-8b-bnb-4bit/blobs/9e6494b882aaa2f3f942

In [15]:
!zip -r nixous_model.zip /content/nixous_model

  adding: content/nixous_model/ (stored 0%)
  adding: content/nixous_model/tokenizer.json (deflated 74%)
  adding: content/nixous_model/tokenizer_config.json (deflated 96%)
  adding: content/nixous_model/adapter_model.safetensors (deflated 7%)
  adding: content/nixous_model/special_tokens_map.json (deflated 71%)
  adding: content/nixous_model/adapter_config.json (deflated 54%)
  adding: content/nixous_model/README.md (deflated 66%)


In [16]:
!zip -r outputs.zip /content/outputs

  adding: content/outputs/ (stored 0%)
  adding: content/outputs/checkpoint-2000/ (stored 0%)
  adding: content/outputs/checkpoint-2000/tokenizer.json (deflated 74%)
  adding: content/outputs/checkpoint-2000/tokenizer_config.json (deflated 96%)
  adding: content/outputs/checkpoint-2000/adapter_model.safetensors (deflated 7%)
  adding: content/outputs/checkpoint-2000/trainer_state.json (deflated 81%)
  adding: content/outputs/checkpoint-2000/scheduler.pt (deflated 56%)
  adding: content/outputs/checkpoint-2000/optimizer.pt (deflated 18%)
  adding: content/outputs/checkpoint-2000/special_tokens_map.json (deflated 71%)
  adding: content/outputs/checkpoint-2000/training_args.bin (deflated 51%)
  adding: content/outputs/checkpoint-2000/adapter_config.json (deflated 54%)
  adding: content/outputs/checkpoint-2000/README.md (deflated 66%)
  adding: content/outputs/checkpoint-2000/rng_state.pth (deflated 25%)
  adding: content/outputs/checkpoint-1500/ (stored 0%)
  adding: content/outputs/check

In [17]:
!zip -r sample_data.zip /content/sample_data

  adding: content/sample_data/ (stored 0%)
  adding: content/sample_data/anscombe.json (deflated 83%)
  adding: content/sample_data/README.md (deflated 42%)
  adding: content/sample_data/california_housing_test.csv (deflated 76%)
  adding: content/sample_data/mnist_test.csv (deflated 88%)
  adding: content/sample_data/mnist_train_small.csv (deflated 88%)
  adding: content/sample_data/california_housing_train.csv (deflated 79%)


In [18]:
!zip -r wandb.zip /content/wandb

  adding: content/wandb/ (stored 0%)
  adding: content/wandb/run-20240527_130203-8tr5q0a6/ (stored 0%)
  adding: content/wandb/run-20240527_130203-8tr5q0a6/logs/ (stored 0%)
  adding: content/wandb/run-20240527_130203-8tr5q0a6/logs/debug.log (deflated 73%)
  adding: content/wandb/run-20240527_130203-8tr5q0a6/logs/debug-internal.log (deflated 95%)
  adding: content/wandb/run-20240527_130203-8tr5q0a6/files/ (stored 0%)
  adding: content/wandb/run-20240527_130203-8tr5q0a6/files/wandb-summary.json (deflated 44%)
  adding: content/wandb/run-20240527_130203-8tr5q0a6/files/output.log (deflated 81%)
  adding: content/wandb/run-20240527_130203-8tr5q0a6/files/wandb-metadata.json (deflated 75%)
  adding: content/wandb/run-20240527_130203-8tr5q0a6/files/requirements.txt (deflated 55%)
  adding: content/wandb/run-20240527_130203-8tr5q0a6/files/config.yaml (deflated 75%)
  adding: content/wandb/run-20240527_130203-8tr5q0a6/run-8tr5q0a6.wandb (deflated 76%)
  adding: content/wandb/run-20240527_130203